In [1]:
import pandas as pd
import seaborn as sns
%matplotlib inline

In [ ]:
data = pd.read_csv('bank_note_data.csv')

In [ ]:
# checking data head
data.head()

In [ ]:
# creating countplot of classes
sns.countplot(x='Class',data=data)

In [ ]:
# creating pairplot of the data
sns.pairplot(data,hue='Class')

Data preparation

In [ ]:
# standard scaling
from sklearn.preprocessing import StandardScaler

In [ ]:
scaler = StandardScaler()

In [ ]:
scaler.fit(data.drop('Class',axis=1))

In [ ]:
scaled_features = scaler.fit_transform(data.drop('Class',axis=1))

In [ ]:
# converting scaled features to a dataframe
df_feat = pd.DataFrame(scaled_features,columns=data.columns[:-1])
df_feat.head()

Train Test Split

In [ ]:
X = df_feat

In [ ]:
y = data['Class']

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3)

Tenserflow

In [ ]:
import tensorflow as tf

In [ ]:
# creating a list of feature column objects
df_feat.columns

In [ ]:
image_var = tf.feature_column.numeric_column("Image.Var")
image_skew = tf.feature_column.numeric_column('Image.Skew')
image_curt = tf.feature_column.numeric_column('Image.Curt')
entropy = tf.feature_column.numeric_column('Entropy')

In [ ]:
feat_cols = [image_var,image_skew,image_curt,entropy]

In [ ]:
classifier = tf.estimator.DNNClassifier(hidden_units=[10, 20, 10], n_classes=2,feature_columns=feat_cols)

In [ ]:
input_func = tf.estimator.inputs.pandas_input_fn(x=X_train,y=y_train,batch_size=20,shuffle=True)

In [ ]:
classifier.train(input_fn=input_func,steps=500)

Model evaluation

In [ ]:
pred_fn = tf.estimator.inputs.pandas_input_fn(x=X_test,batch_size=len(X_test),shuffle=False)

In [ ]:
note_predictions = list(classifier.predict(input_fn=pred_fn))

In [ ]:
note_predictions[0]

In [ ]:
final_preds  = []
for pred in note_predictions:
    final_preds.append(pred['class_ids'][0])

In [ ]:
from sklearn.metrics import classification_report,confusion_matrix

In [ ]:
print(confusion_matrix(y_test,final_preds))

In [ ]:
print(classification_report(y_test,final_preds))